### Imports

In [6]:
# Libraries
import os
from importlib import reload
import keras
import tensorflow as tf
import pandas as pd
import numpy as np

# Scripts
from acml import data, runners, utils

In [7]:
!pip show tensorflow

Name: tensorflow
Version: 2.10.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 


In [8]:
!pip show keras

Name: keras
Version: 2.10.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: c:\users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages
Requires: 
Required-by: keras-tuner, tensorflow


In [9]:
reload(data)

<module 'acml.data' from 'c:\\Users\\sonar\\Desktop\\acml_2024_s2\\runner\\acml\\data.py'>

In [10]:
reload(runners)

<module 'acml.runners' from 'c:\\Users\\sonar\\Desktop\\acml_2024_s2\\runner\\acml\\runners.py'>

## `main`

In [11]:
# Silence information messages from TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Check whether tf is using the GPU
utils.check_gpu_use() 


GPU DETECTED ✓


### Load data

In [12]:
NUM_FOLDS = 10

In [13]:
# FULL DATASET:
# --- Takes _ minutes to run with data in csv --- #
# --- Takes 6 minutes to run with data in npz --- #

fold_dfs = data.get_fold_dfs_with_data(
    fold_definition_csv='data/deepship_5k_seg_3s.csv',
    new_path_to_root='data/csv_deepship_psd_npz',
    npz=True,
    label_encodings={0:'Tanker', 1:'Cargo', 2:'Tug', 3:'Passengership'},
    n_folds=NUM_FOLDS
)
print('Successfully loaded folds with spectrograms.\n')

Successfully loaded folds with spectrograms.



In [40]:
# import cv2
# for df in fold_dfs:
#     df['spectrogram'] = df['spectrogram'].apply(lambda x: cv2.resize(x, (128, 128)))

In [14]:
fold_dfs[0]['spectrogram'][0].shape

(116, 100)

In [15]:
all_data = pd.concat(fold_dfs, ignore_index=True)
all_data

,Cargo,Passengership,Tanker,Tug,spectrogram
0,0,1,0,0,"[[-61.4701110244397, -69.4606846462652, -64.82..."
1,0,1,0,0,"[[-53.0530004443909, -52.0373175079799, -49.98..."
2,0,1,0,0,"[[-70.4963084080672, -68.487216890753, -65.003..."
3,0,1,0,0,"[[-60.1075427126643, -53.2378802812778, -53.97..."
4,0,1,0,0,"[[-45.4194339406413, -47.3021866694952, -47.65..."
...,...,...,...,...,...
28903,0,0,1,0,"[[-44.469157661851, -47.7828338363683, -49.551..."
28904,0,0,1,0,"[[-45.1544400339298, -41.4536694314034, -37.66..."
28905,0,0,1,0,"[[-46.100093830703, -47.9903366244218, -50.018..."
28906,0,0,1,0,"[[-50.131997022499, -48.1243386917267, -44.999..."


In [16]:
classes = ['Cargo', 'Passengership', 'Tanker', 'Tug']

X = all_data['spectrogram'].to_numpy(copy=True)
X = np.stack(X)

# (number of samples, number of time bins, number of freqs, channel)
X = np.expand_dims(X, axis=-1) 

y = all_data[classes].to_numpy(copy=True)

In [17]:
print(X.shape)
print(y.shape)

(28908, 116, 100, 1)
(28908, 4)


In [18]:
0.2*23099

4619.8

In [19]:
X_train = X[:4620]
y_train = y[:4620]

X_test = X[4620:]
y_test = y[4620:]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4620, 116, 100, 1)
(4620, 4)
(24288, 116, 100, 1)
(24288, 4)


### Perform hyperparameter tuning

In [21]:
from keras_tuner import RandomSearch
from models import cnn_lstm
from keras import layers

In [22]:
def build_model(hp):
    input_layer = layers.Input(shape=(116, 100, 1), name='psd')

    # CNN Layers
    conv_2d = cnn_lstm._get_cnn_layers(
        input_layer,
        filter_size=(5, 5),
        num_filters=hp.Choice('num_filters', [64, 128])
    )
    
    layer_norm = layers.LayerNormalization()(conv_2d)
    layer_norm = layers.Reshape([-1, layer_norm.shape[2] * layer_norm.shape[3]])(layer_norm)

    # LSTM Layer
    lstm_units = hp.Choice('lstm_units', [512, 1024])
    fwd_lstm_layer = layers.LSTM(lstm_units, activation='tanh')(layer_norm)

    flatten = layers.Flatten(name="flatten")(fwd_lstm_layer)

    # Dense Layer
    dense_units = hp.Choice('dense_units', [32, 64, 128])
    dense = layers.Dense(dense_units, activation='relu')(flatten)
    
    # Classification Output
    classification = layers.Dense(4, activation="softmax", name="classification")(dense)

    model = keras.Model([input_layer], [classification], name="cnn_lstm")
    
    # Select optimizer and learning rate
    learning_rate = hp.Choice('learning_rate', [1e-5, 1e-4, 1e-3, 1e-2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )    

    return model


In [24]:
# Set up Keras Tuner
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=20,
    executions_per_trial=1,
    directory="tuner_results",
    project_name="cnn_lstm_hyperparameter_tuning"
)

# Start the search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[cnn_lstm.callback_early_stop()])

Trial 20 Complete [00h 19m 38s]
val_accuracy: 0.549489438533783

Best val_accuracy So Far: 0.5661643743515015
Total elapsed time: 02h 27m 38s


In [25]:
tuner.search_space_summary()

Search space summary
Default search space size: 7
filter_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5, 7], 'ordered': True}
num_filters (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64, 128, 256], 'ordered': True}
lstm_units (Choice)
{'default': 256, 'conditions': [], 'values': [256, 512, 1024, 2048], 'ordered': True}
dense_units (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64, 128, 256], 'ordered': True}
dense_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'sgd'], 'ordered': False}
learning_rate (Choice)
{'default': 1e-05, 'conditions': [], 'values': [1e-05, 0.0001, 0.001, 0.01], 'ordered': True}


In [26]:
models = tuner.get_best_models(num_models=5)
models


In [28]:
tuner.results_summary()

Results summary
Results in tuner_results\cnn_lstm_hyperparameter_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 08 summary
Hyperparameters:
filter_size: 5
num_filters: 128
lstm_units: 512
dense_units: 64
dense_activation: relu
optimizer: adam
learning_rate: 1e-05
Score: 0.5661643743515015

Trial 07 summary
Hyperparameters:
filter_size: 3
num_filters: 128
lstm_units: 1024
dense_units: 32
dense_activation: relu
optimizer: adam
learning_rate: 0.001
Score: 0.551342248916626

Trial 17 summary
Hyperparameters:
filter_size: 3
num_filters: 32
lstm_units: 1024
dense_units: 256
dense_activation: tanh
optimizer: sgd
learning_rate: 0.01
Score: 0.5511363744735718

Trial 16 summary
Hyperparameters:
filter_size: 7
num_filters: 64
lstm_units: 512
dense_units: 64
dense_activation: relu
optimizer: sgd
learning_rate: 0.001
Score: 0.5507246255874634

Trial 19 summary
Hyperparameters:
filter_size: 3
num_filters: 256
lstm_units: 2048
dense_units: 64
dense_activation: ta

In [27]:
models[0].summary()

Model: "cnn_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 psd (InputLayer)            [(None, 116, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 116, 100, 128)     3328      
                                                                 
 batch_normalization (BatchN  (None, 116, 100, 128)    512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 116, 100, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 58, 50, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 58, 50, 128)       409